In [ ]:
import cv2 as c
import numpy as np


cap = c.VideoCapture(0)
imgTarget = c.imread('targetimg.jpg')
vid = c.VideoCapture('vid.mp4')


detection = False
frameCounter = 0

success, imgVideo = vid.read()
ht,wt,ct = imgTarget.shape
imgVideo = c.resize(imgVideo,(wt,ht))


orb = c.ORB_create(nfeatures=1000)
kp1,des1 = orb.detectAndCompute(imgTarget,None)
#imgTarget = c.drawKeypoints(imgTarget,kp1,None)


while True:

    success,imgWebcam = cap.read()
    imgAug = imgWebcam.copy()
    kp2,des2 = orb.detectAndCompute(imgWebcam,None)
    if not detection:
        vid.set(c.CAP_PROP_POS_FRAMES,0)
        frameCounter = 0
    else:
        if frameCounter == vid.get(c.CAP_PROP_FRAME_COUNT):
            vid.set(c.CAP_PROP_POS_FRAMES,0)
            frameCounter = 0
        success, imgVideo = vid.read()
        imgVideo = c.resize(imgVideo,(wt,ht))
        
        
    #imgWebcam = c.drawKeypoints(imgWebcam,kp2,None)
    bf = c.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.75 *n.distance:
            good.append(m)
    
    imgFeatures = c.drawMatches(imgTarget,kp1,imgWebcam,kp2,good,None,flags=2)
    if len(good) > 25:
        detection = True
        srcpts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dstpts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
        matrix, mask = c.findHomography(srcpts,dstpts,c.RANSAC,5)
        
        pts = np.float32([[0,0],[0,ht],[wt,ht],[wt,0]]).reshape(-1,1,2)
        dst = c.perspectiveTransform(pts,matrix)
        img2 = c.polylines(imgWebcam,[np.int32(dst)],True,(58,194,106),5)
        
        imgWarp = c.warpPerspective(imgVideo,matrix,(imgWebcam.shape[1],imgWebcam.shape[0]))
        
        maskNew = np.zeros((imgWebcam.shape[0],imgWebcam.shape[1]),np.uint8)
        c.fillPoly(maskNew,[np.int32(dst)],(255,255,255))
        maskInv = c.bitwise_not(maskNew)
        
        imgAug = c.bitwise_and(imgAug,imgAug,mask = maskInv)
        imgAug = c.bitwise_or(imgWarp,imgAug)
    c.imshow('maskNew',imgAug)
    #c.imshow('imgWarp',imgWarp)
    frameCounter += 1
    #c.imshow('img2',img2)
    #c.imshow('imgFeatures',imgFeatures)
    #c.imshow('imgTarget',imgTarget)
    #c.imshow('vid',imgVideo)
    #c.imshow('webcam',imgWebcam)
    c.waitKey(1)
